## 6.5 A quantum repeater
This example extends quantum teleportation over a number of nodes, that can act as a quantum repeater.

In [ ]:
using ImageShow
using StrangelyDisplayed
using StrangelyQuantum

The method `qrepeater` has two aguments:
 - `p` ($0 \le p \le 1$) is the probability density that `q[1]` is `1`
 - `init` is a `Bool` to detemine whether to apply a Pauli X gate to `q[1]`.

In [ ]:
function qrepeater(p, init)
    @assert 0 ≤ p ≤ 1
    simulator = SimpleQuantumExecutionEnvironment()
    program = Program(5)
    # randomly choose whether to set the first bit to 1
    r = rand(Bool)

    # The first steps are the same as in the teleport example
    # if init is true, set the first bit to 1
    if init
        step0 = Step()
        addGate(step0, X(1))
        addStep(program, step0)
    end
    step1 = Step()
    addGate(step1, Hadamard(2))
    addGate(step1, Hadamard(4))
    step2 = Step()
    addGate(step2, Cnot(2, 3))
    addGate(step2, Cnot(4, 5))
    step3 = Step()
    addGate(step3, Cnot(1, 2))
    step4 = Step()
    addGate(step4, Hadamard(1))
    step5 = Step()
    addGate(step5, Measurement(1))
    addGate(step5, Measurement(2))
    step6 = Step()
    addGate(step6, Cnot(2, 3))
    step7 = Step()
    addGate(step7, Cz(1, 3))

    # The following steps represent the repeater element
    step8 = Step()
    addGate(step8, Cnot(3, 4))
    step9 = Step()
    addGate(step9, Hadamard(3))
    step10 = Step()
    addGate(step10, Measurement(3))
    addGate(step10, Measurement(4))
    step11 = Step()
    addGate(step11, Cnot(4, 5))
    step12 = Step()
    addGate(step12, Cz(3, 5))
    addStep(program, step1)
    addStep(program, step2)
    addStep(program, step3)
    addStep(program, step4)
    addStep(program, step5)
    addStep(program, step6)
    addStep(program, step7)
    addStep(program, step8)
    addStep(program, step9)
    addStep(program, step10)
    addStep(program, step11)
    addStep(program, step12)
    initializeQubit(program, 1, p)
    result = runProgram(simulator, program)
    qubits = getQubits(result)
    q3 = qubits[3]
    v3 = measure(q3)
    println("sent: ", Int(init), ", received: ", v3)
    return program
end

As in **Setion 6.5**, we set `p = 0.4`. We execute `qrepeater` with `init = false`:

In [ ]:
program0 = qrepeater(0.4, false);

In [ ]:
drawProgram(program0)

In [ ]:
drawTrialHistogram(program0, 1000)

Inspection of the imges shows that there is an 84% chance that a `1` is received:
$$0.84=1-0.4^2$$.

With the same value of `p`, but with `init = true`, we get

In [ ]:
program1 = qrepeater(0.4, true);

In [ ]:
drawProgram(program1)

In [ ]:
drawTrialHistogram(program1, 1000)

Now we see that there is a 16% chance that a `1` is received:
$$0.16=0.4^2$$.